# Basic Summerizer

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from dotenv import load_dotenv
from langchain.docstore.document import Document
load_dotenv()
from langchain.chains.summarize import load_summarize_chain
from typing import List

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
import re
def load_transcript(url: str) -> str | None:
    """
    Fetch transcript for a YouTube video.
    """
    pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11})'
    match = re.search(pattern, url)
    if match:
        video_id = match.group(1)
        try:
            captions = YouTubeTranscriptApi().fetch(video_id,languages=['en','hi']).snippets
            data = [f"{item.text} ({item.start})" for item in captions]
            return " ".join(data)
        except Exception as e:
            print(f"❌ Error fetching transcript: {e}")
            return None
def text_splitter(transcript: str):
    """
    Splits transcript text into chunks for embeddings.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return splitter.create_documents([transcript])

In [3]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

# ------------------ Structured Schema ------------------
class AnsandTime(BaseModel):
    Summary: List[str] = Field(description="Summarizes the YouTube transcripts  (no timestamps here)")
    timestamps: float = Field(description="The time (in seconds) from where the summary was taken")

structured_model = model.with_structured_output(AnsandTime)
system_prompt = """
You are a concise and efficient summarizer. Your task is to extract the key points from a provided transcript and present them in two ways:

1.  A list of simple bullet points that summarizes the content. Do not include timestamps in this summary.
2.  The exact timestamp (in seconds) of the specific segment of the transcript you are summarizing.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "Transcript:\n{text}")
])


In [4]:
youtube_captions = load_transcript("https://youtu.be/nQa31xdXbGk")
docs =text_splitter(youtube_captions)


In [ ]:
chain = load_summarize_chain(
        llm=structured_model,
        chain_type="map_reduce",   # or "refine", or "stuff" if short
        map_prompt=prompt,
        combine_prompt=prompt
    )
summary_result = chain.run(docs)


C:\Users\prana\AppData\Local\Temp\ipykernel_12712\1609784719.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary_result = chain.run(docs)
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limit

# Summerizer 2

In [18]:
import re
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
def load_transcript(url: str) -> str | None:
    """
    Fetch transcript for a YouTube video.
    """
    pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11})'
    match = re.search(pattern, url)
    if match:
        video_id = match.group(1)
        try:
            captions = YouTubeTranscriptApi().fetch(video_id).snippets
            data = [f"{item.text} ({item.start})" for item in captions]
            return " ".join(data)
        except Exception as e:
            print(f"❌ Error fetching transcript: {e}")
            return None
        

def parse_transcript(transcript):
    """Extract text and timestamps from transcript"""
    pattern = r'(.*?)\((\d+\.?\d*)\)'
    matches = re.findall(pattern, transcript)
    
    segments = []
    for text, timestamp in matches:
        text = text.strip()
        if text:
            segments.append({
                'text': text,
                'timestamp': float(timestamp)
            })
    
    return segments
def format_for_llm(segments):
    """Format segments for LLM input"""
    formatted = []
    for segment in segments:
        formatted.append(f"[{segment['timestamp']}s] {segment['text']}")
    
    return "\n".join(formatted)

In [ ]:
youtube_captions = load_transcript("https://www.youtube.com/watch?v=LPZh9BOjkQs")
segments = parse_transcript(youtube_captions)
formatted_transcript = format_for_llm(segments)
formatted_transcript

In [28]:
def summarize_video(transcript):
    """Main function to summarize video transcript"""
    
    """# Parse transcript
    segments = parse_transcript(transcript)
    if not segments:
        return "No valid transcript segments found"
    
    # Format for LLM
    formatted_transcript = format_for_llm(segments)
    """
    # Initialize LLM
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    
    # Create prompt
    prompt = ChatPromptTemplate.from_template("""
    You are a video summarizer. Analyze this timestamped transcript and provide:

    1. **OVERVIEW**: Brief summary of the video content
    2. **KEY POINTS**: Main points with their timestamps (format: "Point description [timestamp]")
    3. **MAIN TOPICS**: List of topics covered

    Transcript:
    {transcript}

    Format your response clearly with the three sections above.
    Always include timestamps in square brackets for each key point.
    """)
    
    # Generate summary
    response = llm.invoke(prompt.format(transcript=transcript))
    
    return response.content

In [29]:
print(summarize_video(youtube_captions))

Here's a summary of the video content:

---

### 1. OVERVIEW
The video provides a comprehensive explanation of how large language models (LLMs) function, primarily as next-word predictors. It details the two main phases of their training: an initial "pre-training" on vast amounts of text to learn language patterns, followed by "reinforcement learning with human feedback" (RLHF) to align their behavior with user preferences. The explanation also covers the underlying technological advancements, particularly the Transformer architecture and its key components like the attention mechanism, which enable the parallel processing and contextual understanding crucial for modern LLMs, highlighting the immense computational scale involved.

### 2. KEY POINTS
*   Large Language Models (LLMs) are sophisticated mathematical functions that predict what word comes next for any piece of text [33.38].
*   Instead of predicting one word with certainty, LLMs assign a probability to all possible next word

# Summerizer 3 advanced

In [2]:
import re
from typing import List, Dict, Any
from dataclasses import dataclass
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field
import json
from langchain_google_genai import ChatGoogleGenerativeAI
from youtube_transcript_api import YouTubeTranscriptApi
@dataclass
class TimestampedSegment:
    """Represents a segment of transcript with timestamp"""
    text: str
    start_time: float
    end_time: float = None

class SummaryPoint(BaseModel):
    """Individual summary point with timestamp reference"""
    content: str = Field(description="The main point or topic discussed")
    timestamp: float = Field(description="Timestamp in seconds when this topic was discussed")
    importance: str = Field(description="Importance level: high, medium, low")

class BriefSummary(BaseModel):
    key_points: List[SummaryPoint] = Field(description="List of key points with timestamps and also inlcudes brief summary on Each key Point , does not include unimportant topics")

class MainTopic(BaseModel):
    topic: str = Field(description="The topic name")
    timestamp: float = Field(description="Approx timestamp when topic starts")

class VideoSummary(BaseModel):
    """Complete video summary structure"""
    title: str = Field(description="Suggested title for the video based on content")
    overview: str = Field(description="Brief overview of the entire video upto 200 words")
    key_pt_brief_summary : List[BriefSummary] = Field(description="Brief Summary for each Key Point")
    main_topics: List[MainTopic] = Field(description="List of main topics covered")
    duration_summary: str = Field(description="Summary of video duration and pacing")

class YouTubeVideoSummarizer:
    def __init__(self):
       
        self.llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0.3
    )
        self.output_parser = PydanticOutputParser(pydantic_object=VideoSummary)
    
    def parse_transcript(self, transcript: str) -> List[TimestampedSegment]:
        """
        Parse transcript text with timestamps in format:
        "text content (timestamp) more text (timestamp)"
        
        Args:
            transcript: Raw transcript string
            
        Returns:
            List of TimestampedSegment objects
        """
        segments = []
        
        # Regular expression to find text and timestamps
        # Pattern: captures text followed by timestamp in parentheses
        pattern = r'(.*?)\((\d+\.?\d*)\)'
        
        matches = re.findall(pattern, transcript)
        
        for i, (text, timestamp) in enumerate(matches):
            text = text.strip()
            if text:  # Only add non-empty text segments
                segment = TimestampedSegment(
                    text=text,
                    start_time=float(timestamp),
                    end_time=float(matches[i+1][1]) if i+1 < len(matches) else None
                )
                segments.append(segment)
        
        return segments
    
    def format_transcript_for_llm(self, segments: List[TimestampedSegment]) -> str:
        """
        Format parsed segments for LLM processing
        
        Args:
            segments: List of timestamped segments
            
        Returns:
            Formatted transcript string
        """
        formatted_parts = []
        
        for segment in segments:
            timestamp_str = f"[{segment.start_time}s]"
            formatted_parts.append(f"{timestamp_str} {segment.text}")
        
        return "\n".join(formatted_parts)
    
    def create_summary_prompt(self) -> ChatPromptTemplate:
        """Create the prompt template for video summarization"""
        
        system_message = SystemMessagePromptTemplate.from_template(
            """You are an expert video content analyzer and summarizer. 
            You will receive a transcript of a YouTube video with timestamps.
            
            Your task is to:
            1. Create a structured summary with key points
            2. Include timestamp references for each point
            3. Identify main topics and themes
            4. Provide an overall assessment of the video content
            
            Be concise but comprehensive. Focus on the most important information.
            Always include the timestamp reference for where you found each piece of information.
            
            {format_instructions}
            """
        )
        
        human_message = HumanMessagePromptTemplate.from_template(
            """Please analyze and summarize the following timestamped video transcript:

            {transcript}
            
            Topics of the Transcript are provided below:
            {topics}
            
            """
        )
        
        return ChatPromptTemplate.from_messages([system_message, human_message])
    
    def summarize_video(self, transcript: str , topics : str) -> Dict[str, Any]:
        """
        Main method to summarize a video transcript
        
        Args:
            transcript: Raw transcript string with timestamps
            
        Returns:
            Structured summary dictionary
        """
        # Parse the transcript
        segments = self.parse_transcript(transcript)
        
        if not segments:
            return {"error": "No valid transcript segments found"}
        
        # Format for LLM
        formatted_transcript = self.format_transcript_for_llm(segments)
        
        # Create prompt
        prompt = self.create_summary_prompt()
        
        # Prepare the input
        formatted_prompt = prompt.format_prompt(
            transcript=formatted_transcript,
            topics = topics,
            format_instructions=self.output_parser.get_format_instructions()
        )
        
        # Generate summary
        try:
            response = self.llm(formatted_prompt.to_messages())
            parsed_output = self.output_parser.parse(response.content)
            
            # Convert to dictionary and add metadata
            summary_dict = parsed_output.dict()
            summary_dict["total_segments"] = len(segments)
            summary_dict["video_duration"] = max([s.start_time for s in segments]) if segments else 0
            
            return response,parsed_output , summary_dict
            
        except Exception as e:
            return {"error": f"Failed to generate summary: {str(e)}"}
    def format_summary_output(self, summary: Dict[str, Any]) -> str:
        if "error" in summary:
            return f"Error: {summary['error']}"
        
        output = []
        output.append(f"📹 VIDEO SUMMARY")
        output.append("=" * 50)
        output.append(f"Title: {summary['title']}")
        output.append(f"Duration: {summary.get('video_duration', 0):.1f} seconds")
        output.append(f"Total Segments: {summary.get('total_segments', 0)}")
        output.append("")
        
        output.append("📋 OVERVIEW:")
        output.append(summary['overview'])
        output.append("")
        
        output.append("🎯 KEY POINTS:")
        for i, point in enumerate(summary['key_pt_brief_summary'], 1):
            output.append(f"{i}. {point['content']}")
            output.append(f"   ⏰ Timestamp: {point['timestamp']}s | Importance: {point['importance']}")
            output.append("")
        
        output.append("📚 MAIN TOPICS:")
        for topic in summary['main_topics']:
            output.append(f"• {topic}")
        output.append("")
        
        output.append("⏱️ PACING ANALYSIS:")
        output.append(summary['duration_summary'])
        
        return "\n".join(output)
    


In [3]:
topics = """
🎯 Main Topic 1: Introduction to MCP Architecture  ⏰ 76.4
----------------------------------------------------
   🔹 Subtopic 1.1: Simplified Architecture: Host and Server  ⏰ 119.759 high
   🔹 Subtopic 1.2: Defining Host (AI Chatbot)  ⏰ 154.239 None
   🔹 Subtopic 1.3: Defining Server (Tool for specific tasks)  ⏰ 210.879 None
   🔹 Subtopic 1.4: Communication Flow Example (User to GitHub Server)  ⏰ 254.08 None
====================================================

🎯 Main Topic 2: Refining MCP Architecture: Introducing the Client  ⏰ 349.919
----------------------------------------------------
   🔹 Subtopic 2.1: Host does not communicate directly with Server  ⏰ 374.4 None
   🔹 Subtopic 2.2: MCP Client as a Helper  ⏰ 387.52 None
   🔹 Subtopic 2.3: Client's role in communication  ⏰ 418.479 None
   🔹 Subtopic 2.4: One-on-one Client-Server Relationship  ⏰ 530.959 None
   🔹 Subtopic 2.5: Analogy: Phone, SIM, Network  ⏰ 610.72 None
   🔹 Subtopic 2.6: Benefits of Client-Server Architecture (Decoupling, Scalability)  ⏰ 733.279 high
====================================================

🎯 Main Topic 3: Refining MCP Architecture: Primitives  ⏰ 879.12
----------------------------------------------------
   🔹 Subtopic 3.1: Definition of Primitives (Server offerings to Host)  ⏰ 889.199 None
   🔹 Subtopic 3.2: Type 1: Tools (Actions AI can ask Server to perform)  ⏰ 932.079 None
   🔹 Subtopic 3.3: Type 2: Resources (Structured data sources AI can read)  ⏰ 1013.04 None
   🔹 Subtopic 3.4: Type 3: Prompts (Predefined templates to shape AI behavior)  ⏰ 1088.4 None
   🔹 Subtopic 3.5: Standard Operations/Functions for Primitives  ⏰ 1362.24 None
   🔹 Subtopic 3.6: Tools Operations: `tools/list`, `tools/call`  ⏰ 1376.0 None
   🔹 Subtopic 3.7: Resources Operations: `resources/list`, `resources/read`, `resources/subscribe`, `resources/unsubscribe`  ⏰ 1463.919 None
   🔹 Subtopic 3.8: Prompts Operations: `prompts/list`, `prompts/get`  ⏰ 1508.559 None
====================================================

🎯 Main Topic 4: Refining MCP Architecture: Data Layer  ⏰ 1617.84
----------------------------------------------------
   🔹 Subtopic 4.1: Definition of Data Layer (Language and Grammar)  ⏰ 1640.24 None
   🔹 Subtopic 4.2: JSON RPC 2.0 as the Foundation  ⏰ 1713.12 high
   🔹 Subtopic 4.3: What is RPC (Remote Procedure Call)?  ⏰ 1796.48 None
   🔹 Subtopic 4.4: What is JSON RPC?  ⏰ 1939.6 None
   🔹 Subtopic 4.5: JSON RPC Request/Response Structure and Examples  ⏰ 2002.799 None
   🔹 Subtopic 4.6: JSON RPC Examples in MCP (tools, resources, batching, notifications, errors)  ⏰ 2232.56 None
   🔹 Subtopic 4.7: Why JSON RPC over REST API?  ⏰ 2810.72 high
   🔹 Subtopic 4.8: Reason 1: Lightweight Protocol  ⏰ 2912.96 None
   🔹 Subtopic 4.9: Reason 2: Supports Bidirectional Communication  ⏰ 2997.119 None
   🔹 Subtopic 4.10: Reason 3: Transport Agnostic  ⏰ 3040.96 None
   🔹 Subtopic 4.11: Reason 4: Supports Batching  ⏰ 3109.599 None
   🔹 Subtopic 4.12: Reason 5: Supports Notifications  ⏰ 3142.64 None
====================================================

🎯 Main Topic 5: Refining MCP Architecture: Transport Layer  ⏰ 3207.2
----------------------------------------------------
   🔹 Subtopic 5.1: Definition of Transport Layer (Mechanism for moving JSON RPC messages)  ⏰ 3214.24 None
   🔹 Subtopic 5.2: Two Types of Servers: Local and Remote  ⏰ 3328.16 None
   🔹 Subtopic 5.3: Local Servers and STDIO Transport  ⏰ 3571.119 None
   🔹 Subtopic 5.4: STDIO Full Form and Concept (Standard Input/Output)  ⏰ 3576.24 None
   🔹 Subtopic 5.5: How STDIO Transport Works (Host as sub-process, Parent-Child relationship)  ⏰ 3682.319 None
   🔹 Subtopic 5.6: STDIO Demo (Python script example)  ⏰ 3778.4 None
   🔹 Subtopic 5.7: Benefits of STDIO Transport (Fast, Secure, Simple)  ⏰ 3981.359 None
   🔹 Subtopic 5.8: Remote Servers and HTTP + SSE Transport  ⏰ 4056.64 None
   🔹 Subtopic 5.9: HTTP for Remote Servers (POST requests with JSON payload, authentication)  ⏰ 4074.079 None
   🔹 Subtopic 5.10: SSE Full Form and Concept (Server-Sent Events for streaming)  ⏰ 4177.679 None
   🔹 Subtopic 5.11: Revisiting JSON RPC's Transport Agnostic Nature  ⏰ 4317.76 high
====================================================

🎯 Main Topic 6: Final MCP Architecture Summary  ⏰ 4475.199
----------------------------------------------------
   🔹 Subtopic 6.1: Recap of all components (Host, Clients, Servers, Primitives, JSON RPC, Transport Layers)  ⏰ 4488.08 None

"""

In [5]:
def load_transcript(url: str) -> str | None:
    """
    Fetch transcript for a YouTube video.
    """
    pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11})'
    match = re.search(pattern, url)
    if match:
        video_id = match.group(1)
        try:
            captions = YouTubeTranscriptApi().fetch(video_id,languages=['en','hi']).snippets
            data = [f"{item.text} ({item.start})" for item in captions]
            return " ".join(data)
        except Exception as e:
            print(f"❌ Error fetching transcript: {e}")
            return None
sample_transcript =load_transcript("https://youtu.be/nQa31xdXbGk")

# Initialize the summarizer (you'll need to provide your OpenAI API key)
summarizer = YouTubeVideoSummarizer()

# Test transcript parsing
segments = summarizer.parse_transcript(sample_transcript)
print("Parsed Segments:")
for segment in segments:
    print(f"[{segment.start_time}s] {segment.text}")
    break

print("\n" + "="*50)
print("Formatted for LLM:")
formatted = summarizer.format_transcript_for_llm(segments)
print(formatted)

# Generate summary (uncomment when you have API key)
response , parsed_output , summary = summarizer.summarize_video(sample_transcript , topics = topics)


Parsed Segments:
[0.08s] हाय गाइस, माय नेम इज नितेश एंड यू वेलकम

Formatted for LLM:
[0.08s] हाय गाइस, माय नेम इज नितेश एंड यू वेलकम
[2.399s] टू माय YouTube चैनल। इस वीडियो में हम
[5.04s] लोग अपना एमसीपी प्लेलिस्ट कंटिन्यू
[6.879s] करेंगे। सो अगर आपको याद होगा पिछला जो
[9.36s] वीडियो था वहां पर मैंने आपको बहुत डिटेल
[11.519s] में यह समझाया था कि एमसीपी की जरूरत
[14.24s] क्यों है? और आज का जो वीडियो है वो उसी
[17.76s] वीडियो का कंटिन्यूएशन है और आज के वीडियो
[20.24s] में हम व्हाट एस्पेक्ट के ऊपर बात करेंगे
[23.519s] एमसीपी के। मैंने सोचा है कि व्हाट
[26.48s] एस्पेक्ट में मैं मेनली तीन चीजें कवर
[28.8s] करूंगा। पहला एमसीपी का आर्किटेक्चर जो कि
[32.32s] बहुत इंपॉर्टेंट है। दूसरा एमसीपी का लाइफ
[35.84s] साइकिल। क्लाइंट और सर्वर एक दूसरे से
[39.28s] कैसे बात करते हैं? स्टेप बाय स्टेप उस
[41.44s] पूरे प्रोसेस को हम डिकोड करेंगे। एंड
[44.16s] थर्डली हम एमसीपी के कुछ एडवांस्ड
[46.96s] कांसेप्ट्स जो हैं उनको भी टच करेंगे। बट
[50.16s] द प्रॉब्लम इज दिस कि जब मैं यह वीडियो
[52.32s] रिकॉर्ड करने ब

C:\Users\prana\AppData\Local\Temp\ipykernel_28348\2116591000.py:163: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.llm(formatted_prompt.to_messages())
C:\Users\prana\AppData\Local\Temp\ipykernel_28348\2116591000.py:167: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  summary_dict = parsed_output.dict()


In [5]:
type(response) ,  type(parsed_output) , type(summary)

(langchain_core.messages.ai.AIMessage, __main__.VideoSummary, dict)

In [6]:
def format_summary_output(summary: Dict[str, Any]) -> str:
        if "error" in summary:
            return f"Error: {summary['error']}"
        
        output = []
        output.append(f"📹 VIDEO SUMMARY")
        output.append("=" * 50)
        output.append(f"Title: {summary['title']}")
        output.append(f"Duration: {summary.get('video_duration', 0):.1f} seconds")
        output.append(f"Total Segments: {summary.get('total_segments', 0)}")
        output.append("")
        
        output.append("📚 MAIN TOPICS:")
        for topic in summary['main_topics']:
            output.append(f"• {topic['topic']}\t")
            output.append(f"{topic['timestamp']}")
        output.append("")
        
        output.append("📋 OVERVIEW:")
        output.append(summary['overview'])
        output.append("")
        
        output.append("🎯 KEY POINTS:")
        for i, point in enumerate(summary['key_pt_brief_summary'][0]['key_points'],1):
            output.append(f"{i}. {point['content']}")
            output.append(f"   ⏰ Timestamp: {point['timestamp']}s | Importance: {point['importance']}")
            output.append("")
        
        output.append("⏱️ PACING ANALYSIS:")
        output.append(summary['duration_summary'])
        
        return "\n".join(output)
print(format_summary_output(summary))

📹 VIDEO SUMMARY
Title: Deep Dive into the Anthropic's Model Composition Protocol (MCP) Architecture
Duration: 4627.5 seconds
Total Segments: 1642

📚 MAIN TOPICS:
• Introduction to MCP Architecture	
76.4
• Refining MCP Architecture: Introducing the Client	
349.919
• Refining MCP Architecture: Primitives	
879.12
• Refining MCP Architecture: Data Layer	
1617.84
• Refining MCP Architecture: Transport Layer	
3207.2
• Final MCP Architecture Summary	
4475.199

📋 OVERVIEW:
This video provides a comprehensive explanation of Anthropic's Model Composition Protocol (MCP) architecture.  Starting with a simplified model of a host (AI chatbot) and server, the video progressively introduces the client as a crucial intermediary for communication.  The concept of primitives—tools, resources, and prompts—offered by the server to the host is detailed.  A significant portion focuses on the data layer, explaining JSON RPC 2.0 as its foundation and comparing it to REST APIs.  The video then delves into the t

In [ ]:
embed_url ="https://www.youtube.com/embed/LPZh9BOjkQs"
for x in summary['key_points']:
    timestamp_url_play = f"{embed_url}?start={int(float(x['timestamp']))}&autoplay=1"
    print(timestamp_url_play)

In [88]:
import streamlit as  st
embed_url ="https://www.youtube.com/embed/LPZh9BOjkQs"
for i, point in enumerate(summary['key_points'], 1):
    timestamp_url_play = f"{embed_url}?start={int(float(point['timestamp']))}&autoplay=1"
    print(timestamp_url_play)

https://www.youtube.com/embed/LPZh9BOjkQs?start=33&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=37&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=88&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=108&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=127&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=227&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=254&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=276&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=382&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=392&autoplay=1


0.0
120.0
879.0
1362.0
1638.0
1770.0
2002.0
2826.0
3207.0
3571.0
4056.0
4475.0
